In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import joblib

In [2]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


In [3]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score



In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import joblib


In [5]:
# 1. Load Data
train_df = pd.read_csv('x_test.csv')
test_df = pd.read_csv('xy_train.csv')


In [6]:
print("Train columns:", train_df.columns.tolist())
print("Test columns:", test_df.columns.tolist())


Train columns: ['ID', 'text']
Test columns: ['ID', 'text', 'label']


In [7]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

train_df['clean_text'] = train_df['text'].apply(preprocess)
test_df['clean_text'] = test_df['text'].apply(preprocess)


In [8]:
tfidf = TfidfVectorizer(max_features=5000)
X_train = tfidf.fit_transform(train_df['clean_text'])
X_test = tfidf.transform(test_df['clean_text'])


In [9]:
y_test = test_df['label']  # since train.csv has no labels, we assume model is pretrained or you're testing only

# For demonstration, we will train on test set since train.csv has no labels — in practice, use labeled data
clf = LogisticRegression()
clf.fit(X_test, y_test)  # Replace X_test and y_test with X_train, y_train if train labels available



LogisticRegression()

In [10]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.83      0.83      0.83     25798
           1       0.81      0.81      0.81     22015
           2       1.00      0.01      0.01       187

    accuracy                           0.82     48000
   macro avg       0.88      0.55      0.55     48000
weighted avg       0.82      0.82      0.82     48000



In [11]:
joblib.dump(clf, 'fake_news_model.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [12]:
# Load model and vectorizer
import joblib

model = joblib.load("fake_news_model.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

# Preprocess + Predict
def predict(text):
    cleaned = preprocess(text)
    vector = vectorizer.transform([cleaned])
    prob = model.predict_proba(vector)[0][1]
    label = "Fake" if prob > 0.5 else "Real"
    return label, prob

# Example
result, confidence = predict("Breaking: Aliens land in Washington")
print(f"Prediction: {result} (Confidence: {confidence:.2f})")


Prediction: Real (Confidence: 0.12)
